In [ ]:
import numpy as np 

from aiida import load_profile
from aiida.orm import load_node
from aiida_vasp_ext_mlauer.util.structures import gen_aiida_structure

from aiida.plugins import DataFactory

load_profile('lauerm-prod')

Profile<uuid='f4d23725fea446908276e21eedde97d3' name='lauerm-test'>

In [2]:
def generate_wz_structure(alat, clat, species):

    lattice = np.array(
        [[1/2, -np.sqrt(3)/2, 0],
         [1/2,  np.sqrt(3)/2, 0],
         [  0,             0, clat/alat]]
    )

    basis = np.array(
        [[1/3, 2/3, 0.0],
         [2/3, 1/3, 0.5],
         [1/3, 2/3, 0.375],
         [2/3, 1/3, 0.875]]
    )

    aiida_str = gen_aiida_structure(alat, lattice, species, basis)
    aiida_str.store()

    return aiida_str

In [ ]:
STRUCTURE_PKs = [
    1186, # wz-AlN
    1187, # wz-ScN
    1188, # wz-GaN
    1189, # wz-InN
]


INIT_LAT_PARA = [
    (3.112, 4.982), # AlN - Vurgaftman
    (3.715,	4.475), # ScN - https://pmc.ncbi.nlm.nih.gov/articles/PMC12250753/
    (3.189, 5.185), # GaN - Vurgaftman
    (3.545, 5.703), # InN - Vurgaftman
]

SPECIES = [
    ["Al", "Al", "N", "N"],
    ["Sc", "Sc", "N", "N"],
    ["Ga", "Ga", "N", "N"],
    ["In", "In", "N", "N"],
]

In [4]:

for idx in range(4):
    species = SPECIES[idx]
    alat, clat = INIT_LAT_PARA[idx]
    if STRUCTURE_PKs[idx] is None:
        structure = generate_wz_structure(alat, clat, species)
    
    else:
        structure = load_node(STRUCTURE_PKs[idx])
    
    print(f"Hexagonal wz-{"".join(np.unique(species))} is stored at {structure.pk}")

    pmg_str = structure.get_pymatgen()
    pmg_str.to_file(f'../test/{"".join(np.unique(species))}.vasp', fmt='poscar')

Hexagonal wz-AlN is stored at 5250
Hexagonal wz-NSc is stored at 5251
Hexagonal wz-GaN is stored at 5252
Hexagonal wz-InN is stored at 5253


/home/mlauer/anaconda3/envs/aiida/lib/python3.12/site-packages/pymatgen/io/vasp/inputs.py:619: EncodingWarning: We strongly encourage explicit `encoding`, and we would use UTF-8 by default as per PEP 686
  with zopen(filename, mode="wt") as file:
